In [4]:
# For data processing
import numpy as np
import pandas as pd

# For API usage
import requests as r

# For progress bar
from tqdm import tqdm

In [8]:
API_KEY = '96c7131158ec2d199c338e83be8f0fd5'

# Create empty DF to store the movie data
movie = pd.DataFrame()

resp = r.get("https://api.themoviedb.org/3/discover/movie?" \
                 f"api_key={API_KEY}&language=en-US&sort_by=revenue.desc&" \
                 "include_video=false&page=1")
# Find number of pages in the API call
total_pages = resp.json()['total_pages']

# Relevant movie columns
cols = ['title', 'id', 'popularity', 'vote_count', 'vote_average','release_date', 'original_language', 'adult']
# No. of pages to collect responses from
for i in tqdm(range(1, 501)):
        resp = r.get(f"https://api.themoviedb.org/3/discover/movie?api_key={API_KEY}&language=en-US&sort_by=revenue.desc&include_video=false&page={i}")
        
        # Select the relevant columns
        try:
                data = pd.DataFrame(resp.json()['results'],columns=cols)
        except: continue

        movie = movie.append(data)
movie = movie.reset_index(drop=True)

more_cols = ['budget', 'genres', 'revenue', 'year_released', 'decade_released', 'keywords']

rows = []
for i in tqdm(range(len(movie))):
        row = []
        movie_id = movie['id'][i]
        
        
        # Get the movie's other data that is not found in all movies
        resp = r.get(f'https://api.themoviedb.org/3/movie/{movie_id}?api_key={API_KEY}&language=en-US')


        try:
            budget = resp.json()['budget']
            if budget == 0:
                row.append(np.nan)
            else:
                row.append(budget)
        except KeyError:
            row.append(np.nan)
        try:
            genres = []
            for item in resp.json()['genres']:
                genres.append(item['name'])
            row.append(genres)
        except:
            row.append(np.nan)
        
        try:
            revenue = resp.json()['revenue']
            if revenue == 0:
                row.append(np.nan)
            else:
                row.append(revenue)
        except KeyError:
            row.append(np.nan)
        
        try:
            year_movie = int(resp.json()['release_date'][0:4])
            if year_movie >= 2020:
                decade_num = '2020s'
            elif year_movie >= 2010:
                decade_num = '2010s'
            elif year_movie >=2000:
                decade_num = '2000s'
            elif year_movie >=1990:
                decade_num = '1990s'
            elif year_movie >=1980:
                decade_num = '1980s'
            elif year_movie >=1970:
                decade_num = '1970s'
            elif year_movie >=1960:
                decade_num = '1960s'
            else:
                decade_num = 'Before 1960s'
            row.append(year_movie)
            row.append(decade_num)
        except:
            row.append(np.nan)
            row.append(np.nan)
        
        resp = r.get(f'https://api.themoviedb.org/3/movie/{movie_id}/keywords?api_key={API_KEY}')

        keywords = [kw['name'] for kw in resp.json()['keywords']]
        if keywords == []:
            row.append(np.nan)
        else: row.append(keywords)

        rows.append(row)

# Create a DF containing all the data for all movies
stats = pd.DataFrame(rows, columns=more_cols)

df = pd.concat([movie, stats], axis=1)

df


  0%|          | 0/500 [00:00<?, ?it/s]/var/folders/18/95745h4n0hl94h03fr01c1jc0000gn/T/ipykernel_64739/3446272278.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  movie = movie.append(data)
  0%|          | 1/500 [00:00<00:56,  8.78it/s]/var/folders/18/95745h4n0hl94h03fr01c1jc0000gn/T/ipykernel_64739/3446272278.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  movie = movie.append(data)
/var/folders/18/95745h4n0hl94h03fr01c1jc0000gn/T/ipykernel_64739/3446272278.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  movie = movie.append(data)
  1%|          | 3/500 [00:00<02:30,  3.31it/s]/var/folders/18/95745h4n0hl94h03fr01c1jc0000gn/T/ipykernel_64739/3446272278.py:23: FutureWarning: The frame.append method is depreca

,title,id,popularity,vote_count,vote_average,release_date,original_language,adult,budget,genres,revenue,year_released,decade_released,keywords
0,Watch Your Six,1072246,0.600,1,10.0,,es,False,1.0,"[Action, Thriller, Comedy, Drama]",4999999999,NaN,NaN,NaN
1,Avatar,19995,386.424,28849,7.6,2009-12-15,en,False,237000000.0,"[Action, Adventure, Fantasy, Science Fiction]",2920357254,2009.0,2000s,"[culture clash, future, space war, space colon..."
2,Avengers: Endgame,299534,157.992,22959,8.3,2019-04-24,en,False,356000000.0,"[Adventure, Science Fiction, Action]",2799439100,2019.0,2010s,"[space travel, time travel, time machine, sequ..."
3,Condemned,1106123,0.620,1,10.0,,en,False,300.0,"[Horror, Mystery]",2470000000,NaN,NaN,"[moon, killing, detective, journalist, rain, m..."
4,Avatar: The Way of Water,76600,6686.292,6873,7.7,2022-12-14,en,False,350000000.0,"[Science Fiction, Adventure, Action]",2312335665,2022.0,2020s,"[loss of loved one, dying and death, alien lif..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,Maybe This Time,273812,1.280,2,8.5,2014-05-28,tl,False,NaN,"[Romance, Comedy, Drama]",1500000,2014.0,2010s,NaN
9996,Ohm Shanthi Oshaana,257160,4.589,44,7.3,2014-02-07,ml,False,220000.0,"[Comedy, Romance]",1500000,2014.0,2010s,"[poem, rebel, kung fu, nostalgia, doctor, bike..."
9997,Beau Geste,189621,1.869,10,5.9,1926-08-24,en,False,900000.0,"[Action, Adventure, Drama]",1500000,1926.0,Before 1960s,NaN
9998,Il nostro matrimonio è in crisi,173336,1.197,22,5.8,2001-12-31,it,False,4000000.0,[Comedy],1500000,2001.0,2000s,NaN


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   title              10000 non-null  object 
 1   id                 10000 non-null  int64  
 2   popularity         10000 non-null  float64
 3   vote_count         10000 non-null  int64  
 4   vote_average       10000 non-null  float64
 5   release_date       9997 non-null   object 
 6   original_language  10000 non-null  object 
 7   adult              10000 non-null  bool   
 8   budget             7497 non-null   float64
 9   genres             10000 non-null  object 
 10  revenue            10000 non-null  int64  
 11  year_released      9952 non-null   float64
 12  decade_released    9952 non-null   object 
 13  keywords           8992 non-null   object 
dtypes: bool(1), float64(4), int64(3), object(6)
memory usage: 1.0+ MB


In [ ]:
df.to_csv('uncleaned_1015.csv')